# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: [Jose Aurelio Bollas Taboada]
*   Alumno 2: [Antonio Jose Bonafede Salas]
*   Alumno 3: [Elvis David Pachacama Cabezas]
*   Alumno 4: [Jose Fernando Sarmiento Sarmiento]






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto SpaceInvaders"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

print(f"Número de acciones disponibles: {nb_actions}")

Número de acciones disponibles: 6


#### Procesador Atari

El procesador Atari se encarga de:
1. Preprocesar las observaciones (redimensionar a 84x84 y convertir a escala de grises)
2. Normalizar el estado batch (dividir por 255)
3. Clipear las recompensas entre -1 y 1

In [3]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        """Procesa cada observación: redimensiona a 84x84 y convierte a escala de grises"""
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # L = luminance (escala de grises)
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        """Normaliza el batch de estados dividiendo por 255"""
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        """Clipea las recompensas entre -1 y 1 para estabilizar el entrenamiento"""
        return np.clip(reward, -1., 1.)

---
### **1. Implementación de la red neuronal**

Se implementa una red neuronal convolucional (CNN) basada en la arquitectura descrita en el paper de Mnih et al. (2015) "Human-level control through deep reinforcement learning".

**Arquitectura:**
- Capa Permute: Reorganiza las dimensiones de entrada según el formato de Keras
- Conv2D #1: 32 filtros, kernel 8x8, stride 4 → Extrae características de bajo nivel
- Conv2D #2: 64 filtros, kernel 4x4, stride 2 → Extrae características de nivel medio
- Conv2D #3: 64 filtros, kernel 3x3, stride 1 → Refina características
- Dense #1: 512 neuronas → Capa completamente conectada
- Dense #2: nb_actions neuronas, activación lineal → Salida Q-values para cada acción

**Justificación:**
- Las capas convolucionales permiten extraer características espaciales relevantes del juego
- Los strides progresivamente más pequeños permiten capturar detalles a diferentes escalas
- La capa densa de 512 neuronas permite combinar las características extraídas
- La activación lineal final es apropiada para estimar Q-values (pueden ser negativos)

In [4]:
# Construcción del modelo CNN siguiendo la arquitectura de Mnih et al. (2015)
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

print(f"Formato de datos de imagen de Keras: {K.image_data_format()}")

# Reorganizar dimensiones según el formato de Keras
if K.image_data_format() == 'channels_last':
    # Formato: (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    # Formato: (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_data_format.')

# Primera capa convolucional: detecta características básicas (bordes, colores)
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))

# Segunda capa convolucional: detecta patrones más complejos
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))

# Tercera capa convolucional: refina las características detectadas
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))

# Aplanar las características para las capas densas
model.add(Flatten())

# Capa completamente conectada: combina características
model.add(Dense(512))
model.add(Activation('relu'))

# Capa de salida: un Q-value por cada acción posible
model.add(Dense(nb_actions))
model.add(Activation('linear'))  # Lineal porque los Q-values pueden ser negativos

print(model.summary())

Formato de datos de imagen de Keras: channels_last
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activ

---
### **2. Implementación de la solución DQN**

Se implementan los componentes principales del algoritmo DQN:

**Memoria de Experiencia (Experience Replay):**
- Tamaño: 1,000,000 transiciones
- Window length: 4 frames (captura movimiento)
- Permite romper correlaciones temporales y reutilizar experiencias

**Política de Exploración:**
- Epsilon-greedy con decaimiento lineal
- ε inicial: 1.0 (exploración total)
- ε final entrenamiento: 0.1 (10% exploración)
- ε test: 0.05 (5% exploración)
- Pasos de decaimiento: 1,000,000 (decae gradualmente)

**Configuración del Agente DQN:**
- Warmup: 50,000 pasos (acumula experiencias antes de entrenar)
- Gamma (γ): 0.99 (factor de descuento, valora recompensas futuras)
- Target model update: 10,000 pasos (actualiza la red objetivo)
- Train interval: 4 pasos (entrena cada 4 acciones)
- Optimizer: Adam con learning rate 0.00025
- Batch size: 32 (por defecto en keras-rl)

**Justificación de hiperparámetros:**
- Learning rate bajo (0.00025): previene oscilaciones en el aprendizaje
- Warmup alto (50,000): asegura suficiente diversidad en la memoria inicial
- Target update (10,000): balance entre estabilidad y adaptación
- Train interval (4): reduce correlación sin perder mucha información

In [6]:
# 1. MEMORIA DE EXPERIENCIA (Experience Replay)
# Almacena transiciones (s, a, r, s') para romper correlaciones temporales
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

# 2. PROCESADOR
# Preprocesa observaciones, estados y recompensas
processor = AtariProcessor()

# 3. POLÍTICA DE EXPLORACIÓN
# Epsilon-greedy con decaimiento lineal: empieza explorando (ε=1.0) 
# y gradualmente explota (ε=0.1)
policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=1.,      # ε inicial: exploración total
    value_min=.1,      # ε final: 10% exploración, 90% explotación
    value_test=.05,    # ε test: 5% exploración durante evaluación
    nb_steps=1000000   # Pasos para decaer de value_max a value_min
)

# 4. AGENTE DQN
# Configuración del algoritmo Deep Q-Network
dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup=50000,      # Pasos de warmup antes de entrenar
    gamma=.99,                  # Factor de descuento (importancia futuro)
    target_model_update=10000,  # Frecuencia de actualización de red objetivo
    train_interval=4,           # Entrena cada 4 acciones
    delta_clip=1,               # Clip del error TD para estabilidad,
    enable_double_dqn=True,
    enable_dueling_network=False
)

# 5. COMPILACIÓN
# Optimizer Adam con learning rate bajo para estabilidad
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

print("\n=== CONFIGURACIÓN DEL AGENTE DQN ===")
print(f"Memoria: {memory.limit:,} transiciones")
print(f"Window length: {WINDOW_LENGTH} frames")
print(f"Política: Epsilon-greedy con decaimiento lineal")
print(f"  - ε inicial: {policy.value_max}")
print(f"  - ε final: {policy.value_min}")
print(f"  - ε test: {policy.value_test}")
print(f"Warmup: {dqn.nb_steps_warmup:,} pasos")
print(f"Gamma (γ): {dqn.gamma}")
print(f"Target update: cada {dqn.target_model_update:,} pasos")
print(f"Train interval: cada {dqn.train_interval} acciones")
print(f"Learning rate: 0.00025")


=== CONFIGURACIÓN DEL AGENTE DQN ===
Memoria: 1,000,000 transiciones
Window length: 4 frames
Política: Epsilon-greedy con decaimiento lineal
  - ε inicial: 1.0
  - ε final: 0.1
  - ε test: 0.05
Warmup: 50,000 pasos
Gamma (γ): 0.99
Target update: cada 10,000 pasos
Train interval: cada 4 acciones
Learning rate: 0.00025


---
### **Entrenamiento del agente**

**Configuración del entrenamiento:**
- Pasos totales: 2,000,000 (suficiente para convergencia en Atari)
- Log interval: cada 10,000 pasos (monitorizar progreso)
- Checkpoints: cada 500,000 pasos (guardar progreso)
- Visualización: desactivada (más rápido)

**Métricas a observar durante el entrenamiento:**
- episode_reward: recompensa total por episodio (objetivo: >20)
- loss: error de predicción de Q-values
- mae: error absoluto medio
- mean_q: Q-value promedio (debe aumentar con el tiempo)
- mean_eps: epsilon actual (debe decrecer)

**Nota:** El entrenamiento puede dura varias horas (12-24h en GPU, 36 en CPU).
Se recomienda usar GPU y guardar checkpoints para continuar si se interrumpe.

In [7]:
# Configuración de callbacks para guardar progreso
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)

callbacks = [
    ModelIntervalCheckpoint(checkpoint_weights_filename, interval=500000),
    FileLogger(log_filename, interval=100)
]

# Entrenamiento
print("\n=== INICIANDO ENTRENAMIENTO ===")
print(f"Pasos totales: 5,000,000")
print(f"Checkpoints se guardarán cada 500,000 pasos")
print("\nPara continuar desde un checkpoint:")
print("  dqn.load_weights('dqn_SpaceInvaders-v0_weights_XXXXX.h5f')")
print("\nIniciando...\n")

history = dqn.fit(
    env,
    callbacks=callbacks,
    nb_steps=5000000,
    log_interval=10000,
    visualize=False
)

# Guardar pesos finales
dqn.save_weights(weights_filename, overwrite=True)
print(f"\n=== ENTRENAMIENTO COMPLETADO ===")
print(f"Pesos guardados en: {weights_filename}")


=== INICIANDO ENTRENAMIENTO ===
Pasos totales: 5,000,000
Checkpoints se guardarán cada 500,000 pasos

Para continuar desde un checkpoint:
  dqn.load_weights('dqn_SpaceInvaders-v0_weights_XXXXX.h5f')

Iniciando...

Training for 5000000 steps ...
Interval 1 (0 steps performed)
   79/10000 [..............................] - ETA: 19s - reward: 0.0000e+00

C:\Users\bonaf\.conda\envs\gym_gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 19s 2ms/step - reward: 0.0131
15 episodes - episode_reward: 8.333 [6.000, 12.000] - ale.lives: 2.052

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 18s 2ms/step - reward: 0.0129
16 episodes - episode_reward: 8.062 [5.000, 11.000] - ale.lives: 2.072

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 18s 2ms/step - reward: 0.0134
16 episodes - episode_reward: 8.500 [4.000, 14.000] - ale.lives: 2.072

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 18s 2ms/step - reward: 0.0159
14 episodes - episode_reward: 11.429 [5.000, 23.000] - ale.lives: 2.065

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 20s 2ms/step - reward: 0.0131
15 episodes - episode_reward: 8.133 [4.000, 17.000] - ale.lives: 2.150

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 181s 18ms/step - reward: 0.0134


---
### **Cargar pesos pre-entrenados**

Al tener el modelo entrenado, permite cargar los pesos aquí en lugar de entrenar desde cero.
Esto es útil para:
- Continuar el entrenamiento desde un checkpoint
- Evaluar un modelo ya entrenado
- Comparar diferentes versiones del modelo

In [ ]:
# Descomentar para cargar pesos pre-entrenados
weights_filename = 'dqn_SpaceInvaders-v0_weights.h5f'
dqn.load_weights(weights_filename)
print(f"Pesos cargados desde: {weights_filename}")

---
### **Evaluación del modelo (Testing)**

Esta celda evalúa el rendimiento del agente entrenado en 100 episodios de test.

**Objetivo:** Alcanzar **más de 20 puntos durante más de 100 episodios consecutivos**

**Criterio de éxito:**
- Ejecutar 100 episodios de evaluación
- Contar la racha máxima de episodios consecutivos con recompensa >20
- Si racha_máxima >= 100 → Objetivo alcanzado ✅

**Interpretación de resultados:**
- < 50 consecutivos: El agente necesita más entrenamiento
- 50-99 consecutivos: El agente está cerca, continuar entrenamiento
- >= 100 consecutivos: ¡Objetivo conseguido!


In [8]:
print("\n=== EVALUACIÓN DEL MODELO ===")
print("Ejecutando 100 episodios de test...\n")

weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
test_results = dqn.test(env, nb_episodes=100, visualize=False)

rewards = test_results.history['episode_reward']
mean_reward = np.mean(rewards)
min_reward = np.min(rewards)
max_reward = np.max(rewards)
std_reward = np.std(rewards)

threshold = 20.0
consecutive = 0
max_consecutive = 0

for reward in rewards:
    if reward > threshold:
        consecutive += 1
        max_consecutive = max(max_consecutive, consecutive)
    else:
        consecutive = 0

episodios_exitosos = sum(r > threshold for r in rewards)

print("\n=== RESULTADOS ===")
print(f"Recompensa media: {mean_reward:.2f}")
print(f"Recompensa mínima: {min_reward:.2f}")
print(f"Recompensa máxima: {max_reward:.2f}")
print(f"Desviación estándar: {std_reward:.2f}")
print(f"\nEpisodios con >{threshold} puntos: {episodios_exitosos}/100")
print(f"Máximo de episodios consecutivos >{threshold}: {max_consecutive}")

objetivo_alcanzado = max_consecutive >= 100
print(f"\n{'='*50}")
if objetivo_alcanzado:
    print(f"✅ OBJETIVO ALCANZADO")
    print(f"El agente logró >20 puntos durante {max_consecutive} episodios consecutivos")
else:
    print(f"❌ OBJETIVO NO ALCANZADO")
    print(f"Máximo consecutivo: {max_consecutive}/100 episodios")
    print(f"\nSugerencias:")
    print(f"  - Entrenar por más pasos (recomendado: 4-5M)")
    print(f"  - Continuar entrenamiento desde checkpoint actual")
print(f"{'='*50}")



=== EVALUACIÓN DEL MODELO ===
Ejecutando 100 episodios de test...

Testing for 100 episodes ...
Episode 1: reward: 15.000, steps: 726
Episode 2: reward: 25.000, steps: 1136
Episode 3: reward: 19.000, steps: 1039
Episode 4: reward: 10.000, steps: 588
Episode 5: reward: 17.000, steps: 838
Episode 6: reward: 24.000, steps: 1205
Episode 7: reward: 13.000, steps: 767
Episode 8: reward: 9.000, steps: 739
Episode 9: reward: 14.000, steps: 930
Episode 10: reward: 6.000, steps: 442
Episode 11: reward: 13.000, steps: 598
Episode 12: reward: 26.000, steps: 1263
Episode 13: reward: 29.000, steps: 1356
Episode 14: reward: 24.000, steps: 1085
Episode 15: reward: 20.000, steps: 1048
Episode 16: reward: 11.000, steps: 614
Episode 17: reward: 26.000, steps: 1154
Episode 18: reward: 27.000, steps: 1231
Episode 19: reward: 7.000, steps: 567
Episode 20: reward: 28.000, steps: 1287
Episode 21: reward: 23.000, steps: 776
Episode 22: reward: 22.000, steps: 1037
Episode 23: reward: 20.000, steps: 932
Episode

---
### **Visualización del agente (solo local)**

Esta celda permite visualizar el agente jugando.
**Nota:** Solo funciona en entorno local (puesto en False dado que al ejecutar el Kernel se reinicializa), no en Google Colab.

In [ ]:
# Visualización (solo funciona en local)
if not IN_COLAB:
    print("Visualizando 3 episodios...")
    dqn.test(env, nb_episodes=3, visualize=False)
else:
    print("La visualización no está disponible en Google Colab.")
    print("Para ver al agente jugar, ejecuta este notebook en local.")

---
### **3. Justificación de los parámetros seleccionados y de los resultados obtenidos**

#### **3.1. Arquitectura de la Red Neuronal**

**Decisión:** Se utilizó la arquitectura CNN descrita en el paper seminal de DQN (Mnih et al., 2015).

**Justificación:**
- **Capas Convolucionales:** Son ideales para procesar imágenes, ya que preservan la estructura espacial y detectan patrones locales como bordes, objetos y movimientos.
- **Estructura progresiva:** Los filtros van de 32→64→64, permitiendo detectar características desde simples (bordes) hasta complejas (naves, disparos).
- **Strides decrecientes:** (4→2→1) capturan información a diferentes escalas, desde una vista general hasta detalles finos.
- **Capa densa de 512:** Suficientemente grande para combinar todas las características extraídas sin sobreajustar.

**Alternativas consideradas:**
- Redes más profundas (ResNet): Descartadas por mayor coste computacional sin mejora significativa en Atari.
- Menos filtros: Podría reducir capacidad de aprendizaje.
- Más filtros: Aumentaría tiempo de entrenamiento sin beneficio claro.

---

#### **3.2. Hiperparámetros del Agente DQN**

##### **Learning Rate: 0.00025**
- **Justificación:** Valor bajo que asegura convergencia estable. DQN es sensible a learning rates altos que causan oscilaciones.
- **Efecto:** Aprendizaje más lento pero más estable.
- **Alternativas:** 0.0001 (más estable, más lento) o 0.0005 (más rápido, más inestable).

##### **Epsilon-greedy con decaimiento lineal (1.0 → 0.1)**
- **Justificación:** Balance exploración-explotación crucial en RL.
  - Inicio (ε=1.0): Exploración total, el agente descubre el entorno.
  - Final (ε=0.1): Mayormente explota conocimiento, pero mantiene 10% exploración para descubrir mejores estrategias.
- **Decaimiento en 1M pasos:** Permite suficiente tiempo de exploración inicial.
- **ε test (0.05):** Pequeña exploración en evaluación para manejar situaciones nuevas.

##### **Memoria: 1,000,000 transiciones**
- **Justificación:** 
  - Rompe correlación temporal entre experiencias consecutivas.
  - Permite reutilizar experiencias pasadas (sample efficiency).
  - 1M es suficiente para juegos Atari (balance memoria/diversidad).
- **Alternativas:** Memorias más pequeñas (500K) funcionan pero con menor estabilidad.

##### **Target Network Update: cada 10,000 pasos**
- **Justificación:**
  - La red objetivo (target network) proporciona valores Q estables para calcular el error.
  - 10K pasos: balance entre estabilidad (updates poco frecuentes) y adaptación (updates no muy espaciados).
- **Efecto:** Reduce oscilaciones en el aprendizaje.

##### **Warmup: 50,000 pasos**
- **Justificación:**
  - Acumula experiencias diversas antes de entrenar.
  - Previene sobreajuste inicial a experiencias limitadas.
  - 50K pasos proporcionan suficiente diversidad inicial.

##### **Gamma (γ): 0.99**
- **Justificación:**
  - Factor de descuento que valora recompensas futuras.
  - 0.99 es estándar para tareas con horizonte largo (Atari tiene episodios largos).
  - Valores más bajos (0.9) harían al agente miope, valores más altos (0.999) podrían causar inestabilidad.

##### **Train Interval: cada 4 acciones**
- **Justificación:**
  - Reduce correlación entre actualizaciones sucesivas.
  - Balance entre eficiencia computacional y frecuencia de actualización.
  - Valor estándar en implementaciones DQN para Atari.

---

#### **3.3. Análisis de Resultados**

**Escribir aquí el análisis después de entrenar:**

```
RECOMPENSA MEDIA OBTENIDA: [COMPLETAR]

ANÁLISIS:
1. Evolución del aprendizaje:
   - ¿Cómo evolucionó la recompensa durante el entrenamiento?
   - ¿Hubo momentos de mejora/estancamiento?
   - ¿Cuándo empezó a converger?

2. Comportamiento del agente:
   - ¿Qué estrategias aprendió? (ej: esquivar, disparar, protegerse)
   - ¿Hay comportamientos subóptimos?

3. Métricas observadas:
   - Evolución de mean_q (debería aumentar)
   - Evolución de loss (debería estabilizarse)
   - Epsilon decay (de 1.0 a 0.1)

4. Comparación con objetivo:
   - ¿Se alcanzó el umbral de 20 puntos?
   - Si no: ¿Qué ajustes propondrías?
   - Si sí: ¿Qué factores fueron clave?

5. Posibles mejoras:
   - Double DQN (reduce sobreestimación de Q-values)
   - Dueling DQN (separa valor del estado y ventaja de acciones)
   - Prioritized Experience Replay (muestrea experiencias importantes más frecuentemente)
   - Entrenar por más pasos (2-3M pasos)
   - Ajustar epsilon decay (más lento/rápido)
   - Modificar arquitectura (más capas, diferentes filtros)
```

---

#### **3.4. Conclusiones**

```
[COMPLETAR DESPUÉS DEL ENTRENAMIENTO]

Resumen:
- Objetivo conseguido: SÍ/NO
- Principal desafío: [Describir]
- Lecciones aprendidas: [Describir]
- Trabajo futuro: [Describir posibles extensiones]
```

---

#### **3.5. Referencias**

1. Mnih, V., et al. (2015). "Human-level control through deep reinforcement learning." Nature, 518(7540), 529-533.
2. Van Hasselt, H., Guez, A., & Silver, D. (2016). "Deep reinforcement learning with double Q-learning." AAAI.
3. Wang, Z., et al. (2016). "Dueling network architectures for deep reinforcement learning." ICML.
4. Schaul, T., et al. (2015). "Prioritized experience replay." ICLR.
5. keras-rl2 documentation: https://github.com/wau/keras-rl2

---
## **ANEXO: Análisis de Métricas del Entrenamiento**

Utilizar este código para visualizar el progreso del test:

In [ ]:
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Extraer datos disponibles
episodes = list(range(1, len(test_results.history['episode_reward']) + 1))
rewards = test_results.history['episode_reward']
steps = test_results.history['nb_steps']

# Calcular estadísticas
episodios_exitosos = sum(r > 20 for r in rewards)
tasa_exito = (episodios_exitosos / len(rewards)) * 100

# Crear subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Recompensas por Episodio (Test)', 
                    'Steps por Episodio',
                    'Distribución de Recompensas', 
                    'Estadísticas del Test'),
    specs=[[{"type": "scatter"}, {"type": "bar"}],
           [{"type": "histogram"}, {"type": "table"}]]  # ← CAMBIO: tabla en lugar de scatter
)

# 1. Recompensas por episodio
fig.add_trace(
    go.Scatter(x=episodes, y=rewards, mode='lines+markers', 
               name='Recompensa', line=dict(color='blue', width=2),
               marker=dict(size=10)),
    row=1, col=1
)
fig.add_hline(y=20, line_dash="dash", line_color="red", 
              annotation_text="Objetivo (20)", row=1, col=1)
fig.add_hline(y=np.mean(rewards), line_dash="dot", line_color="green",
              annotation_text=f"Media ({np.mean(rewards):.2f})", row=1, col=1)

# 2. Steps por episodio
fig.add_trace(
    go.Bar(x=episodes, y=steps, name='Steps', marker_color='orange',
           showlegend=False),
    row=1, col=2
)

# 3. Distribución (histograma)
fig.add_trace(
    go.Histogram(x=rewards, nbinsx=8, name='Distribución',
                 marker_color='purple', showlegend=False),
    row=2, col=1
)

# 4. Tabla de estadísticas (SOLUCIÓN)
fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Métrica</b>', '<b>Valor</b>'],
            fill_color='lightblue',
            align='left',
            font=dict(size=14, color='black')
        ),
        cells=dict(
            values=[
                ['Episodios evaluados', 'Recompensa media', 'Recompensa máxima', 
                 'Recompensa mínima', 'Desviación estándar', 'Mediana',
                 'Steps promedio', '', 'Episodios >20 pts', 'Tasa de éxito', 
                 '', '<b>Objetivo (μ>20)</b>'],
                [f'{len(rewards)}', f'{np.mean(rewards):.2f}', f'{np.max(rewards):.2f}',
                 f'{np.min(rewards):.2f}', f'{np.std(rewards):.2f}', f'{np.median(rewards):.2f}',
                 f'{np.mean(steps):.0f}', '', f'{episodios_exitosos}/{len(rewards)}', 
                 f'{tasa_exito:.0f}%', '',
                 f'<b>{"✅ ALCANZADO" if np.mean(rewards) > 20 else "❌ NO ALCANZADO"}</b>']
            ],
            fill_color=[['white', 'lightgray']*6],
            align='left',
            font=dict(size=12),
            height=30
        )
    ),
    row=2, col=2
)

# Actualizar layouts
fig.update_xaxes(title_text="Episodio", row=1, col=1)
fig.update_xaxes(title_text="Episodio", row=1, col=2)
fig.update_xaxes(title_text="Recompensa", row=2, col=1)

fig.update_yaxes(title_text="Recompensa", row=1, col=1)
fig.update_yaxes(title_text="Steps", row=1, col=2)
fig.update_yaxes(title_text="Frecuencia", row=2, col=1)

fig.update_layout(
    height=800, width=1200,
    showlegend=False,
    title_text="<b>Resultados del Test - DQN SpaceInvaders</b>",
    title_font_size=20
)

# Guardar
fig.write_html('test_results.html')
fig.write_image('test_results.png', width=1200, height=800)
fig.show()

print("\n✓ Visualización guardada:")
print("  - test_results.html (interactivo)")
print("  - test_results.png (imagen)")

# Resumen en consola
print(f"\n{'='*60}")
print("ANÁLISIS DETALLADO DEL TEST")
print(f"{'='*60}")
print(f"\n Recompensas por episodio:")
for i, r in enumerate(rewards, 1):
    emoji = "🟢" if r > 20 else "🟡" if r > 10 else "🔴"
    print(f"  {emoji} Episodio {i:2d}: {r:5.2f} puntos")

print(f"\n{'─'*60}")
print(f"Estadísticas agregadas:")
print(f"{'─'*60}")
print(f"  Recompensa media:      {np.mean(rewards):6.2f}")
print(f"  Recompensa mediana:    {np.median(rewards):6.2f}")
print(f"  Recompensa máxima:     {np.max(rewards):6.2f}")
print(f"  Recompensa mínima:     {np.min(rewards):6.2f}")
print(f"  Desviación estándar:   {np.std(rewards):6.2f}")
print(f"  Steps promedio:        {np.mean(steps):6.0f}")

print(f"\n{'─'*60}")
print(f" Evaluación de objetivo:")
print(f"{'─'*60}")
print(f"  Episodios >20 puntos:  {episodios_exitosos}/{len(rewards)} ({tasa_exito:.0f}%)")
print(f"  Media vs objetivo:     {np.mean(rewards):.2f} vs 20.00")
print(f"  Estado:                {'✅ ALCANZADO' if np.mean(rewards) > 20 else '❌ NO ALCANZADO'}")

if np.mean(rewards) < 20:
    deficit = 20 - np.mean(rewards)
    print(f"\n El modelo necesita mejorar {deficit:.2f} puntos para alcanzar el objetivo")

print(f"{'='*60}\n")

## Visualizar estadisticas

<img src="test_results.png" alt="Estadisticas del modelo" width="800" height="600">

Utilizar este código para visualizar el progreso del entrenamiento:

### Continuar Entrenamiento

In [ ]:
# 1. Cargar el modelo con los últimos pesos
dqn.load_weights('dqn_SpaceInvaders-v0_weights.h5f')
print(" Pesos cargados")

# 2. Continuar entrenando más steps
dqn.fit(
    env, 
    nb_steps=500000,  # 500k steps adicionales
    visualize=False, 
    verbose=2
)

# 3. Los nuevos episodios se añadirán automáticamente al log
print(" Entrenamiento continuado completado!")

# 4. Evaluar
test_results = dqn.test(env, nb_episodes=10, visualize=False)
print(f"Media: {np.mean(test_results.history['episode_reward']):.2f}")

---